In [1]:
template_network = TemplateNetwork()

NameError: name 'TemplateNetwork' is not defined

In [1]:
from libspn.generation.dynamic import TemplateNetwork, TopNetwork
from libspn.graph.weights import Weights

template_network = TemplateNetwork()

ix = template_network.add_ivs("iv_x", num_vars=1, num_vals=2)
iy = template_network.add_ivs("iv_y", num_vars=1, num_vals=2)
iz = template_network.add_ivs("iv_z", num_vars=1, num_vals=2)

mixture_x0_w = Weights(num_weights=2, name="mixture_x0_w")
mixture_x0 = template_network.add_sum("mix_x0", inputs=ix, weights=mixture_x0_w)
mixture_x1_w = Weights(num_weights=2, name="mixture_x1_w")
mixture_x1 = template_network.add_sum("mix_x1", inputs=ix, weights=mixture_x1_w)

mixture_y0_w = Weights(num_weights=2, name="mixture_y0_w")
mixture_y0 = template_network.add_sum("mix_y0", inputs=iy, weights=mixture_y0_w)
mixture_y1_w = Weights(num_weights=2, name="mixture_y1_w")
mixture_y1 = template_network.add_sum("mix_y1", inputs=iy, weights=mixture_y1_w)

mixture_z0_w = Weights(num_weights=2, name="mixture_z0_w")
mixture_z0 = template_network.add_sum("mix_z0", inputs=iz, weights=mixture_z0_w)
mixture_z1_w = Weights(num_weights=2, name="mixture_z1_w")
mixture_z1 = template_network.add_sum("mix_z1", inputs=iz, weights=mixture_z1_w)

interface_forward_declaration = [TemplateNetwork.InterfaceNode("Interface0", 0),
                                 TemplateNetwork.InterfaceNode("Interface1", 1)]


mixture_in0_w = Weights(num_weights=2, name="mixture_in0_w")
mixture_in0 = template_network.add_sum(
    "mix_interface0", interface=interface_forward_declaration, weights=mixture_i)
mixture_in1_w = Weights(num_weights=2, name="mixture_in1_w")
mixture_in1 = template_network.add_sum(
    "mix_interface1", interface=interface_forward_declaration, weights=mixture_in1_w)

prod0 = template_network.add_product(
    "prod0", inputs=[mixture_x0, mixture_y0, mixture_z0, mixture_in0], is_root=True)
prod1 = template_network.add_product(
    "prod1", inputs=[mixture_x1, mixture_y1, mixture_z1, mixture_in1], is_root=True)

top_network = TopNetwork()
top_weights = Weights(num_weights=2, name="top_w")
top_root = top_network.add_sum("top_root", interface=interface_forward_declaration,
                               is_root=True, weights=top_weights)

template_steps = template_network.build_n_steps(3)
top_steps = top_network.build(template_steps)

In [2]:
from libspn.inference.value import Value
import libspn as spn
top_root_t2 = top_steps[-1].root_nodes[0]
val = Value(spn.InferenceType.MARGINAL)
val_op = val.get_value(top_root_t2)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


[WARNING] [tensorflow:warning] From /home/jos/spn/libspn/libspn/graph/product.py:133: calling reduce_prod (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
import libspn as spn
spn.display_spn_graph(top_steps[0].root_nodes[0])

# Dynamic SPN with in-graph looping
Code below shows an alternative graph that is more intuitive to build. Perhaps the `DynamicValue` should not really be a special class for this. Also, we still need to support `MPEPath` for in-graph looping. The current state of the code is not yet complete, but it at least shows that this is a feasible option.

Still need to extend it with:
- Providing specific gather indices for the interface nodes. Currently just assumes that it takes the whole node output always. Easy to fix.
- Adding MPE path construction
- Decide on wheter to use a special class `DynamicIVs` or to pass a flag to the `IVs` constructor. More convenient would be to handle the 'dynamicness' at the level of a `VarNode`.

In [1]:
import sys
sys.path.append("../../")
from libspn.graph.ivs import DynamicIVs
from libspn.graph.sum import Sum
from libspn.graph.product import Product
from libspn.graph.node import DynamicInterface
from libspn.inference.value import DynamicValue, DynamicLogValue
import libspn as spn
import tensorflow as tf

In [2]:
tf.reset_default_graph()
max_steps = 4

ix = DynamicIVs(name="iv_x", num_vars=1, num_vals=2, max_steps=max_steps)
iy = DynamicIVs(name="iv_y", num_vars=1, num_vals=2, max_steps=max_steps)
iz = DynamicIVs(name="iv_z", num_vars=1, num_vals=2, max_steps=max_steps)

# First define template network
mix_x0 = Sum(ix, name="mixture_x0")
mix_x1 = Sum(ix, name="mixture_x1")
mix_y0 = Sum(iy, name="mixture_y0")
mix_y1 = Sum(iy, name="mixture_y1")
mix_z0 = Sum(iz, name="mixture_z0")
mix_z1 = Sum(iz, name="mixture_z1")

# Define interface network
intf0 = DynamicInterface(name="interface0")
intf1 = DynamicInterface(name="interface1")

mix_int0 = Sum(intf0, intf1, name="mixture_intf0", interface_head=True)
mix_int1 = Sum(intf0, intf1, name="mixture_intf1", interface_head=True)

# Define template heads
prod0 = Product(mix_x0, mix_y0, mix_z0, mix_int0, name="prod0")
prod1 = Product(mix_x1, mix_y1, mix_z1, mix_int1, name="prod1")

# Register sources for interface nodes
intf0.set_source(prod0)
intf1.set_source(prod1)

# Define top network
top_net = Sum(prod0, prod1, name="top")

spn.generate_weights(root=top_net, init_value=1)

dval = DynamicValue(spn.InferenceType.MARGINAL)
val_op, val_per_step = dval.get_value(top_net, return_sequence=True)

dlogval = DynamicLogValue(spn.InferenceType.MARGINAL)
val_op, val_per_step = dlogval.get_value(top_net, return_sequence=True)

init = spn.initialize_weights(top_net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


[WARNING] [tensorflow:warning] From /home/jos/spn/libspn/libspn/graph/product.py:133: calling reduce_prod (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


[WARNING] [tensorflow:warning] From /home/jos/spn/libspn/libspn/utils/math.py:277: calling reduce_max (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


[WARNING] [tensorflow:warning] From /home/jos/spn/libspn/libspn/utils/math.py:281: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
pathgen = spn.MPEPath(log=True, dynamic=True)
pathgen.get_mpe_path(top_net)

Instructions for updating:
Use the `axis` argument instead


[WARNING] [tensorflow:warning] From /home/jos/spn/libspn/libspn/graph/sum.py:299: calling argmax (from tensorflow.python.ops.math_ops) with dimension is deprecated and will be removed in a future version.
Instructions for updating:
Use the `axis` argument instead


In [6]:
import numpy as np
batch_size = 2
num_vals = 2

xfeed = np.random.randint(num_vals, size=batch_size * max_steps).reshape((max_steps, batch_size, 1))
yfeed = np.random.randint(num_vals, size=batch_size * max_steps).reshape((max_steps, batch_size, 1))
zfeed = np.random.randint(num_vals, size=batch_size * max_steps).reshape((max_steps, batch_size, 1))
xfeed = np.concatenate([np.ones_like(xfeed[:, 0:1]) * -1, xfeed], axis=1)
yfeed = np.concatenate([np.ones_like(xfeed[:, 0:1]) * -1, yfeed], axis=1)
zfeed = np.concatenate([np.ones_like(xfeed[:, 0:1]) * -1, zfeed], axis=1)

count_outs = {}
with tf.Session() as sess:
    sess.run(init)
    val, val_p_s = sess.run([val_op, val_per_step], feed_dict={ix: xfeed, iy: yfeed, iz: zfeed})
    
    for k, v in pathgen._counts.items():
        count_outs[k] = sess.run(v, feed_dict={ix: xfeed, iy: yfeed, iz: zfeed})
print(val_p_s)

print(1/8, 1/64, 1/(8*64), 1/(64 * 64))

print(np.log([1/8, 1/64, 1/(8*64), 1/(64 * 64)]))

[[[ 0.       ]
  [-2.0794415]
  [-2.0794415]]

 [[ 0.       ]
  [-4.158883 ]
  [-4.158883 ]]

 [[ 0.       ]
  [-6.2383246]
  [-6.2383246]]

 [[ 0.       ]
  [-8.317766 ]
  [-8.317766 ]]]
0.125 0.015625 0.001953125 0.000244140625
[-2.07944154 -4.15888308 -6.23832463 -8.31776617]


In [7]:
for k, v in count_outs.items():
    print(k.name)
    print(v)

mixture_z1
[[0.]
 [0.]
 [0.]]
Weights/mixture_z0_Weights
[[4. 0.]
 [4. 0.]
 [2. 2.]]
iv_x
[[4. 0.]
 [1. 3.]
 [4. 0.]]
mixture_x1
[[0.]
 [0.]
 [0.]]
Weights/mixture_y0_Weights
[[4. 0.]
 [2. 2.]
 [1. 3.]]
mixture_y1
[[0.]
 [0.]
 [0.]]
prod0
[[4.]
 [4.]
 [4.]]
top
[[1.]
 [1.]
 [1.]]
mixture_x0
[[4.]
 [4.]
 [4.]]
interface0
[[3.]
 [3.]
 [3.]]
Weights/mixture_x0_Weights
[[4. 0.]
 [1. 3.]
 [4. 0.]]
Weights/mixture_z1_Weights
[[0. 0.]
 [0. 0.]
 [0. 0.]]
mixture_y0
[[4.]
 [4.]
 [4.]]
Weights/mixture_y1_Weights
[[0. 0.]
 [0. 0.]
 [0. 0.]]
Weights/top_Weights
[[1. 0.]
 [1. 0.]
 [1. 0.]]
mixture_z0
[[4.]
 [4.]
 [4.]]
prod1
[[0.]
 [0.]
 [0.]]
iv_y
[[4. 0.]
 [2. 2.]
 [1. 3.]]
mixture_intf1
[[0.]
 [0.]
 [0.]]
Weights/mixture_intf1_Weights
[[0. 0.]
 [0. 0.]
 [0. 0.]]
interface1
[[0.]
 [0.]
 [0.]]
Weights/mixture_x1_Weights
[[0. 0.]
 [0. 0.]
 [0. 0.]]
mixture_intf0
[[3.]
 [3.]
 [3.]]
iv_z
[[4. 0.]
 [4. 0.]
 [2. 2.]]
Weights/mixture_intf0_Weights
[[3. 0.]
 [3. 0.]
 [3. 0.]]


In [8]:
spn.display_tf_graph()

In [8]:
spn.display_spn_graph(top_steps[1].root_nodes[0])

In [9]:
spn.display_spn_graph(top_steps[2].root_nodes[0])
print(top_steps[2].root_nodes[0].is_valid())

True
